<a href="https://colab.research.google.com/github/DmtrRm/IAAC2024_tutorials/blob/main/slice_and_scale_satImagesipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image, ImageFile
import numpy as np

In [2]:
# upload image
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Allow PIL to open very large images
Image.MAX_IMAGE_PIXELS = None  # Removing the limit entirely, use with caution
ImageFile.LOAD_TRUNCATED_IMAGES = True  # To prevent issues with truncated images


In [4]:
def slice_image(image_path, target_width, target_height, current_mpp, target_mpp):
    """
    Slices an image into smaller segments, with progress output.
    Arguments:
    - image_path: Path to the large image file.
    - target_width: The desired width of each slice in pixels.
    - target_height: The desired height of each slice in pixels.
    - current_mpp: Current meters per pixel of the image.
    - target_mpp: Target meters per pixel for output slices.
    """
    # Open the image
    image = Image.open(image_path)

    # Calculate scale factor
    scale_factor = current_mpp / target_mpp

    # Resize image according to the scale factor
    new_width = int(image.width * scale_factor)
    new_height = int(image.height * scale_factor)
    resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)

    # Number of slices in each dimension
    x_slices = new_width // target_width
    y_slices = new_height // target_height
    total_slices = x_slices * y_slices

    # List to hold the image slices
    images = []
    print(f"Total slices to be created: {total_slices}")

    # Progress counter
    slice_count = 0

    for i in range(y_slices):
        for j in range(x_slices):
            left = j * target_width
            upper = i * target_height
            right = left + target_width
            lower = upper + target_height

            # Crop the image to create a slice
            img_slice = resized_image.crop((left, upper, right, lower))
            images.append(img_slice)

            # Optionally, save the slice
            img_slice.save(f'slice_{i}_{j}.png')

            # Update progress
            slice_count += 1
            print(f"Processed {slice_count} of {total_slices} slices...")

    return images

In [5]:
def save_images_to_drive(images, output_dir, quality=90):
    for idx, image in enumerate(images):
        image_path = f"{output_dir}/slice_{idx}.jpg"
        image.save(image_path, 'JPEG', quality=quality)  # Ensure the format is JPEG if using quality
        print(f"Saved {image_path} at quality {quality}")


In [6]:
image_path = "/content/drive/MyDrive/iaac2024/ViennaSatImages/35_3_op_2023.jpg"
target_width = 640  # width of each slice in pixels
target_height = 640 # height of each slice in pixels
current_mpp = 0.15   # current meters per pixel
target_mpp = 0.15    # target meters per pixel

sliced_images = slice_image(image_path, target_width, target_height, current_mpp, target_mpp)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/iaac2024/ViennaSatImages/35_3_op_2023.jpg'

In [ ]:
# have a look at an image
sliced_images[0]

In [ ]:
# Output directory on Google Drive
output_dir = '/content/drive/MyDrive/iaac2024/ViennaSatImages/SlicedImages'

# Save the slices to Google Drive
save_images_to_drive(sliced_images, output_dir)